In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('train.zip', compression='zip', sep='\t')
df

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38,target
0,73.0,25.0,63.0,0.0,-3.307791,59.0,36.0,19544.0,0.0,91.0,...,0.0,0,44211.0,31.0,390.414429,0.571132,0.000,12527.0,0.0,0
1,64.0,37.0,68.0,0.0,-2.527839,106.0,36.0,1457.0,0.0,180.0,...,6.0,0,5600.0,3.0,663.702500,1.447663,0.100,10276.0,0.0,0
2,73.0,34.0,80.0,0.0,-2.985949,72.0,44.0,13071.0,0.0,151.0,...,1.0,0,250889.0,8.0,NaN,0.234987,11.240,4588.0,0.0,0
3,170.0,99.0,203.0,0.0,-3.392264,228.0,NaN,7000.0,NaN,388.0,...,2.0,0,NaN,NaN,471.623097,NaN,NaN,22968.0,0.0,0
4,65.0,20.0,77.0,0.0,-2.657559,115.0,18.0,21725.0,0.0,190.0,...,13.0,1,0.0,0.0,411.162400,0.000000,29.402,15687.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265169,2.0,2.0,1.0,0.0,-2.806880,5.0,NaN,0.0,NaN,7.0,...,1.0,0,NaN,NaN,297.153600,NaN,NaN,2658.0,0.0,0
1265170,15.0,6.0,12.0,0.0,-2.971577,14.0,15.0,0.0,0.0,26.0,...,2.0,0,NaN,NaN,56.500000,NaN,NaN,4694.0,0.0,0
1265171,240.0,135.0,356.0,0.0,-3.313476,401.0,35.0,689921.0,0.0,668.0,...,3.0,0,38000.0,4.0,552.291581,0.114097,25.567,183997.0,205.0,0
1265172,17.0,2.0,6.0,0.0,-3.871151,22.0,NaN,500.0,NaN,40.0,...,4.0,0,NaN,NaN,1475.000000,NaN,NaN,21792.0,0.0,0


In [3]:
df.duplicated().sum()

0

In [4]:
df.columns

Index(['graph_1', 'graph_2', 'graph_3', 'feature_1', 'bki_1', 'graph_4',
       'feature_2', 'graph_5', 'feature_3', 'graph_6',
       ...
       'graph_22', 'application_2', 'bki_14', 'bki_15', 'feature_37', 'bki_16',
       'bki_17', 'mb_15', 'feature_38', 'target'],
      dtype='object', length=101)

In [5]:
# # mb
# df = df[[col for col in df.columns if not col.startswith('mb_')]]
# df

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265174 entries, 0 to 1265173
Columns: 101 entries, graph_1 to target
dtypes: float64(98), int64(3)
memory usage: 974.9 MB


In [7]:
Counter(df.target)

Counter({0: 1167963, 1: 97211})

In [8]:
df.isna().sum().sum()

23039322

In [9]:
# # df.fillna(df.mean(), inplace=True)
# df.fillna(df.median(), inplace=True)
# df.isna().sum().sum()

In [10]:
# df1 = df[df.target == 1]
# df1

In [11]:
# df0 = df[df.target == 0]
# df0

In [12]:
# # удаление выбросов
# for col in df.columns[:-1]:
#     mean = df1[col].mean()
#     std = df1[col].std()
#     threshold = 3
#     df1 = df1[(df1[col] < mean + threshold * std) & (df1[col] > mean - threshold * std)]

# df1

In [13]:
# # удаление выбросов
# for col in df.columns[:-1]:
#     mean = df0[col].mean()
#     std = df0[col].std()
#     threshold = 3
#     df0 = df0[(df0[col] < mean + threshold * std) & (df0[col] > mean - threshold * std)]

# df0

In [14]:
# df = pd.concat([df1, df0], axis=0)
# df

In [15]:
Counter(df.target)

Counter({0: 1167963, 1: 97211})

In [16]:
test = pd.read_csv('test.zip', compression='zip', sep='\t', index_col=0)
test

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,feature_36,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38
0,23.0,29.0,88.0,0.0,-1.976491,64.0,22.0,136633.46,0.0,181.0,...,968.63,4.0,0,0.0,0.0,161.438333,0.969675,30.495,8504.0,0.0
1,95.0,70.0,147.0,0.0,-3.282326,188.0,34.0,268502.00,0.0,305.0,...,243878.17,4.0,0,0.0,0.0,677.439361,0.555632,0.000,109387.0,0.0
2,92.0,52.0,130.0,0.0,-3.815265,135.0,34.0,205182.00,0.0,224.0,...,581071.10,1.0,0,500000.0,3.0,994.984760,0.324038,9.578,175186.0,23.0
3,31.0,8.0,53.0,0.0,-1.658297,81.0,31.0,17315.00,0.0,132.0,...,12833.17,2.0,0,68905.0,8.0,237.651296,0.539998,0.000,33381.0,0.0
4,129.0,69.0,204.0,0.0,-3.543214,271.0,38.0,194450.00,0.0,396.0,...,9334.91,40.0,2,3274000.0,9.0,98.262211,0.168971,8.983,14512.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301484,33.0,8.0,40.0,0.0,-2.709187,75.0,NaN,0.00,0.0,92.0,...,57633.49,3.0,0,NaN,NaN,243.179283,NaN,NaN,93819.0,0.0
301485,16.0,7.0,12.0,0.0,-2.714078,11.0,60.0,0.00,0.0,31.0,...,NaN,3.0,1,243000.0,10.0,NaN,0.198806,0.000,12451.0,NaN
301486,219.0,90.0,267.0,0.0,-3.679674,288.0,36.0,53270.00,0.0,435.0,...,0.00,5.0,0,NaN,NaN,NaN,NaN,NaN,16620.0,0.0
301487,46.0,19.0,51.0,0.0,-3.333991,44.0,55.0,5033.00,0.0,83.0,...,528157.43,0.0,0,NaN,NaN,1956.138630,NaN,NaN,19779.0,56.0


In [17]:
# # mb
# test = test[[col for col in test.columns if not col.startswith('mb_')]]
# test

In [18]:
test.isna().sum().sum()

5104604

In [19]:
# # test.fillna(test.mean(), inplace=True)
# test.fillna(df.median(), inplace=True)
# test.isna().sum().sum()

In [20]:
X = df.drop(['target'], axis=1)
X

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,feature_36,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38
0,73.0,25.0,63.0,0.0,-3.307791,59.0,36.0,19544.0,0.0,91.0,...,27329.01,0.0,0,44211.0,31.0,390.414429,0.571132,0.000,12527.0,0.0
1,64.0,37.0,68.0,0.0,-2.527839,106.0,36.0,1457.0,0.0,180.0,...,2654.81,6.0,0,5600.0,3.0,663.702500,1.447663,0.100,10276.0,0.0
2,73.0,34.0,80.0,0.0,-2.985949,72.0,44.0,13071.0,0.0,151.0,...,0.00,1.0,0,250889.0,8.0,NaN,0.234987,11.240,4588.0,0.0
3,170.0,99.0,203.0,0.0,-3.392264,228.0,NaN,7000.0,NaN,388.0,...,73101.58,2.0,0,NaN,NaN,471.623097,NaN,NaN,22968.0,0.0
4,65.0,20.0,77.0,0.0,-2.657559,115.0,18.0,21725.0,0.0,190.0,...,10279.06,13.0,1,0.0,0.0,411.162400,0.000000,29.402,15687.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265169,2.0,2.0,1.0,0.0,-2.806880,5.0,NaN,0.0,NaN,7.0,...,7428.84,1.0,0,NaN,NaN,297.153600,NaN,NaN,2658.0,0.0
1265170,15.0,6.0,12.0,0.0,-2.971577,14.0,15.0,0.0,0.0,26.0,...,113.00,2.0,0,NaN,NaN,56.500000,NaN,NaN,4694.0,0.0
1265171,240.0,135.0,356.0,0.0,-3.313476,401.0,35.0,689921.0,0.0,668.0,...,342420.78,3.0,0,38000.0,4.0,552.291581,0.114097,25.567,183997.0,205.0
1265172,17.0,2.0,6.0,0.0,-3.871151,22.0,NaN,500.0,NaN,40.0,...,1475.00,4.0,0,NaN,NaN,1475.000000,NaN,NaN,21792.0,0.0


In [21]:
# scaler = StandardScaler()
# X = pd.DataFrame(data=scaler.fit_transform(X))
# X

In [22]:
# pca = PCA(n_components=30)
# X = pd.DataFrame(data=pca.fit_transform(X))
# X

In [23]:
y = df.target
y

0          0
1          0
2          0
3          0
4          0
          ..
1265169    0
1265170    0
1265171    0
1265172    0
1265173    1
Name: target, Length: 1265174, dtype: int64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# knn

In [23]:
knn = KNeighborsRegressor(n_jobs=-1)

In [35]:
knn.fit(X, y)

KNeighborsRegressor(n_jobs=-1)

In [37]:
#Проведем оценку модели
# mae = mean_absolute_error(knn.predict(X_test), y_test)
# mse = mean_squared_error(knn.predict(X_test), y_test)
# r2 = r2_score(knn.predict(X_test), y_test)
# print(mae,mse,r2)

# catboost

In [ ]:
cat = CatBoostRegressor(iterations=1000000, od_wait=1000, task_type='GPU')
cat.fit(X_train, y_train, verbose=200, eval_set=(X_test, y_test))

Learning rate set to 0.01831
0:	learn: 0.2660841	test: 0.2660862	best: 0.2660862 (0)	total: 98.8ms	remaining: 1d 3h 27m 4s
200:	learn: 0.2561422	test: 0.2563576	best: 0.2563576 (200)	total: 7.71s	remaining: 10h 38m 59s
400:	learn: 0.2549258	test: 0.2552880	best: 0.2552880 (400)	total: 16s	remaining: 11h 6m 33s
600:	learn: 0.2542992	test: 0.2548214	best: 0.2548214 (600)	total: 24.3s	remaining: 11h 14m 2s
800:	learn: 0.2538267	test: 0.2545283	best: 0.2545283 (800)	total: 33.4s	remaining: 11h 35m 9s
1000:	learn: 0.2534233	test: 0.2543085	best: 0.2543085 (1000)	total: 41.8s	remaining: 11h 35m 20s
1200:	learn: 0.2530740	test: 0.2541444	best: 0.2541444 (1200)	total: 50.5s	remaining: 11h 40m 6s
1400:	learn: 0.2527640	test: 0.2540235	best: 0.2540235 (1400)	total: 59.2s	remaining: 11h 43m 12s
1600:	learn: 0.2524702	test: 0.2539149	best: 0.2539149 (1600)	total: 1m 7s	remaining: 11h 44m 32s
1800:	learn: 0.2522041	test: 0.2538307	best: 0.2538307 (1800)	total: 1m 16s	remaining: 11h 45m 43s
2000:	le

In [26]:
# cat = CatBoostRegressor()

# grid = {'depth':[None, 7, 8, 9], 
#         'iterations':[15000],
#         'od_wait':[1000],
#         'task_type':['GPU']}

# cat.grid_search(grid,
#                 X=X,
#                 y=y,
#                 verbose=200,
#                 train_size=0.8)

In [27]:
# cat = CatBoostRegressor(iterations=15000, task_type='GPU')
# cat.fit(X, y, verbose=200, eval_set=(X_test, y_test))

In [28]:
#Проведем оценку модели
mae = mean_absolute_error(cat.predict(X_test), y_test)
mse = mean_squared_error(cat.predict(X_test), y_test)
r2 = r2_score(cat.predict(X_test), y_test)
print(mae,mse,r2)

0.12889679063528067 0.0639343212612849 -8.474866748478995


# lgbm

In [38]:
lgbm = LGBMRegressor(n_jobs=-1)

In [39]:
lgbm.fit(X_train, y_train, eval_set=(X_test, y_test))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.251659 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16281
[LightGBM] [Info] Number of data points in the train set: 1252522, number of used features: 85
[LightGBM] [Info] Start training from score 0.076836


LGBMRegressor(n_jobs=-1)

In [40]:
#Проведем оценку модели
mae = mean_absolute_error(lgbm.predict(X_test), y_test)
mse = mean_squared_error(lgbm.predict(X_test), y_test)
r2 = r2_score(lgbm.predict(X_test), y_test)
print(mae,mse,r2)

0.1304420688332594 0.06523148443473198 -11.376591365380863


# voting ensemle

In [19]:
vot = VotingRegressor(
    estimators=[
        ('knn', KNeighborsRegressor(n_jobs=-1)),
        ('catboost', CatBoostRegressor(verbose=0)),
        ('lgbm', LGBMRegressor(n_jobs=-1)),
    ],
    voting='hard',
    n_jobs=-1,
    verbose=True
)
vot.fit(X_train, y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_jobs=-1)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x000001BEDFA60B50>),
                             ('lgbm',
                              LGBMClassifier(class_weight={0: 0.5416156162481175,
                                                           1: 6.5073602781578215},
                                             n_jobs=-1))],
                 n_jobs=-1, verbose=True)

In [20]:
mae = mean_absolute_error(vot.predict(X_test), y_test)
mse = mean_squared_error(vot.predict(X_test), y_test)
r2 = r2_score(vot.predict(X_test), y_test)
print(mae,mse,r2)

(0.26700022742779167, 0.7452576667720519)

# create submit

In [29]:
test

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,feature_36,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38
0,23.0,29.0,88.0,0.0,-1.976491,64.0,22.0,136633.46,0.0,181.0,...,968.63,4.0,0,0.0,0.0,161.438333,0.969675,30.495,8504.0,0.0
1,95.0,70.0,147.0,0.0,-3.282326,188.0,34.0,268502.00,0.0,305.0,...,243878.17,4.0,0,0.0,0.0,677.439361,0.555632,0.000,109387.0,0.0
2,92.0,52.0,130.0,0.0,-3.815265,135.0,34.0,205182.00,0.0,224.0,...,581071.10,1.0,0,500000.0,3.0,994.984760,0.324038,9.578,175186.0,23.0
3,31.0,8.0,53.0,0.0,-1.658297,81.0,31.0,17315.00,0.0,132.0,...,12833.17,2.0,0,68905.0,8.0,237.651296,0.539998,0.000,33381.0,0.0
4,129.0,69.0,204.0,0.0,-3.543214,271.0,38.0,194450.00,0.0,396.0,...,9334.91,40.0,2,3274000.0,9.0,98.262211,0.168971,8.983,14512.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301484,33.0,8.0,40.0,0.0,-2.709187,75.0,NaN,0.00,0.0,92.0,...,57633.49,3.0,0,NaN,NaN,243.179283,NaN,NaN,93819.0,0.0
301485,16.0,7.0,12.0,0.0,-2.714078,11.0,60.0,0.00,0.0,31.0,...,NaN,3.0,1,243000.0,10.0,NaN,0.198806,0.000,12451.0,NaN
301486,219.0,90.0,267.0,0.0,-3.679674,288.0,36.0,53270.00,0.0,435.0,...,0.00,5.0,0,NaN,NaN,NaN,NaN,NaN,16620.0,0.0
301487,46.0,19.0,51.0,0.0,-3.333991,44.0,55.0,5033.00,0.0,83.0,...,528157.43,0.0,0,NaN,NaN,1956.138630,NaN,NaN,19779.0,56.0


In [30]:
sub = pd.read_csv('sample_submission.csv', index_col=0)
sub

,target
0,0
1,0
2,0
3,0
4,0
...,...
301484,0
301485,0
301486,0
301487,0


In [31]:
X_pred = test
# X_pred = scaler.transform(X_pred)
# X_pred = pca.fit_transform(X_pred)
pred =cat.predict(X_pred)
sub['target'] = pred
sub

,target
0,0.046390
1,0.110872
2,0.133951
3,0.065449
4,0.100240
...,...
301484,0.043146
301485,0.041848
301486,0.060838
301487,0.005965


In [32]:
sub.to_csv('sub.csv')